In [0]:
%sql
USE mc_labs;

## calcular a duração da viagem em minutos e depois selecionar apenas as viagens com duração superior a um certo limite.

In [0]:
%sql
WITH TripDurations AS (
  SELECT
    tpep_pickup_datetime,
    tpep_dropoff_datetime,
    trip_distance,
    fare_amount,
    pickup_zip,
    dropoff_zip,
    (UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 60 AS trip_duration_minutes
  FROM
    trips_raw
  WHERE
    tpep_pickup_datetime IS NOT NULL AND tpep_dropoff_datetime IS NOT NULL
)
SELECT
  tpep_pickup_datetime,
  trip_duration_minutes,
  fare_amount,
  pickup_zip
FROM
  TripDurations
WHERE
  trip_duration_minutes > 60 -- Viagens com mais de 60 minutos
ORDER BY
  trip_duration_minutes DESC
LIMIT 20;

## Usando múltiplas CTEs encadeadas para lógica mais complexa. Vamos calcular a duração média da viagem por pickup_zip e depois ranquear esses pickup_zips com base na duração média.

In [0]:
%sql
WITH TripDurations AS (
  SELECT
    pickup_zip,
    (UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 60 AS trip_duration_minutes
  FROM
    trips_raw
  WHERE
    tpep_pickup_datetime IS NOT NULL AND tpep_dropoff_datetime IS NOT NULL AND pickup_zip IS NOT NULL
),
AvgDurationPerZip AS (
  SELECT
    pickup_zip,
    AVG(trip_duration_minutes) AS average_trip_duration_minutes
  FROM
    TripDurations
  GROUP BY
    pickup_zip
)
SELECT
  pickup_zip,
  average_trip_duration_minutes,
  RANK() OVER (ORDER BY average_trip_duration_minutes DESC) AS rank_by_avg_duration
FROM
  AvgDurationPerZip
ORDER BY
  rank_by_avg_duration
LIMIT 20;

## View para viagens com duração calculada. View que adiciona a coluna trip_duration_minutes à tabela trips_raw.

In [0]:
%sql
CREATE OR REPLACE VIEW trips_with_duration AS
SELECT
  tpep_pickup_datetime,
  tpep_dropoff_datetime,
  trip_distance,
  fare_amount,
  pickup_zip,
  dropoff_zip,
  (UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 60 AS trip_duration_minutes
FROM
  trips_raw
WHERE
  tpep_pickup_datetime IS NOT NULL AND tpep_dropoff_datetime IS NOT NULL;

-- Consultar a View
SELECT *
FROM trips_with_duration
WHERE trip_duration_minutes > 120 -- Viagens com mais de 2 horas
LIMIT 10;

## View que combina informações de trips_raw e zip_code_info. View que junta as informações das viagens com os nomes das cidades de partida e destino.

In [0]:
%sql
CREATE OR REPLACE VIEW trips_with_zip_details AS
SELECT
  t.tpep_pickup_datetime,
  t.tpep_dropoff_datetime,
  t.trip_distance,
  t.fare_amount,
  t.pickup_zip,
  z_pickup.city_name AS pickup_city_name,
  t.dropoff_zip,
  z_dropoff.city_name AS dropoff_city_name
FROM
  trips_raw AS t
LEFT JOIN
  zip_code_info AS z_pickup ON t.pickup_zip = z_pickup.zip
LEFT JOIN
  zip_code_info AS z_dropoff ON t.dropoff_zip = z_dropoff.zip
WHERE
  t.pickup_zip IS NOT NULL AND t.dropoff_zip IS NOT NULL;

-- Consultar a View
SELECT
  pickup_city_name,
  COUNT(*) AS total_trips,
  AVG(fare_amount) AS average_fare
FROM
  trips_with_zip_details
GROUP BY
  pickup_city_name
ORDER BY
  total_trips DESC
LIMIT 10;

In [0]:
%sql
-- Cria ou substitui a View 'vw_daily_city_pair_summary' no esquema mc_labs
CREATE OR REPLACE VIEW mc_labs.vw_daily_city_pair_summary AS
WITH
-- CTE 1: Adiciona os nomes das cidades de origem e destino às viagens
-- e filtra viagens com fare_amount > 10 para considerar apenas viagens "significativas"
trips_with_city_names AS (
    SELECT
        -- Removido tr.trip_id, pois não existe na tabela trips_raw
        CAST(tr.tpep_pickup_datetime AS DATE) AS trip_date, -- Usando tpep_pickup_datetime
        tr.fare_amount,
        pz.city_name AS pickup_city_name,
        dz.city_name AS dropoff_city_name
    FROM
        mc_labs.trips_raw AS tr
    LEFT JOIN
        mc_labs.zip_code_info AS pz ON tr.pickup_zip = pz.zip
    LEFT JOIN
        mc_labs.zip_code_info AS dz ON tr.dropoff_zip = dz.zip
    WHERE
        tr.fare_amount > 10 -- Filtra viagens com fare_amount acima de 10
),
-- CTE 2: Filtra as viagens da CTE anterior para que ocorram apenas em dias de semana
-- (segunda a sexta-feira) e garante que as cidades não sejam nulas
weekday_significant_trips AS (
    SELECT
        trip_date,
        fare_amount,
        pickup_city_name,
        dropoff_city_name
    FROM
        trips_with_city_names
    WHERE
        DAYOFWEEK(trip_date) BETWEEN 2 AND 6 -- 2=Segunda-feira, 6=Sexta-feira (Databricks)
        AND pickup_city_name IS NOT NULL -- Garante que temos nomes de cidades válidos
        AND dropoff_city_name IS NOT NULL
)
-- Query final que define a View: Agrega os dados por par de cidades e data
-- para calcular a média do fare_amount e o número de viagens
SELECT
pickup_city_name,
dropoff_city_name,
trip_date,
AVG(fare_amount) AS average_fare,
COUNT(*) AS number_of_trips -- Usando COUNT(*) para contar as linhas (viagens)
FROM
weekday_significant_trips
GROUP BY
pickup_city_name,
dropoff_city_name,
trip_date
ORDER BY
trip_date, pickup_city_name, dropoff_city_name;

In [0]:
%sql
SELECT * FROM mc_labs.vw_daily_city_pair_summary